In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import theano.tensor as T

In [3]:
from fuel.datasets.cifar10 import CIFAR10
from fuel.transformers import ScaleAndShift, Cast, Flatten
from fuel.streams import DataStream
from fuel.schemes import SequentialScheme, ShuffledScheme

CIFAR10.default_transformers = (
    (ScaleAndShift, [2.0 / 255.0, -1], {'which_sources': 'features'}),
    (Cast, [np.float32], {'which_sources': 'features'}) ,
    (Flatten, [], {'which_sources': 'features'}) )

cifar_train = CIFAR10(("train",), subset=slice(None,40000))
cifar_train_stream = DataStream.default_stream(
    cifar_train, iteration_scheme=ShuffledScheme(cifar_train.num_examples, 100))
                                               
cifar_validation = CIFAR10(("train",), subset=slice(40000, None))
cifar_validation_stream = DataStream.default_stream(
    cifar_validation, iteration_scheme=SequentialScheme(cifar_validation.num_examples, 250))

cifar_test = CIFAR10(("test",))
cifar_test_stream = DataStream.default_stream(
    cifar_test, iteration_scheme=SequentialScheme(cifar_test.num_examples, 250))

In [4]:
from modules.Network import Network
from modules.layers.SoftMaxLayer import SoftMaxLayer
from modules.layers.AffineLayer import AffineLayer
from modules.layers.ConvolutionalPoolLayer import ConvolutionalPoolLayer
from modules.initializers.Constant import Constant
from modules.initializers.Normal import Normal

In [5]:
ReLU = lambda x: T.maximum(0.0, x)

net = Network([ConvolutionalPoolLayer(image_shape=(3,32,32), filter_shape=(16,3,5,5), pool_shape=(2,2), conv_mode='same'),
               ConvolutionalPoolLayer(image_shape=(16,16,16), filter_shape=(16,16,5,5), pool_shape=(2,2), conv_mode='same'),
               ConvolutionalPoolLayer(image_shape=(16,8,8), filter_shape=(32,16,5,5), pool_shape=(2,2), conv_mode='same'),
               AffineLayer(32*4*4, 128, activation=ReLU),
               SoftMaxLayer(128, 10, weights_initializer=Normal(std=0.08)) ])

train_cost = net.train(cifar_train_stream, cifar_validation_stream, 30, 8e-3, 0.95, 1e-3)
semilogy(train_cost[:,0], train_cost[:,1], label='batch train cost')
legend()

ValueError: total size of new array must be unchanged
Apply node that caused the error: Reshape{4}(DownsampleFactorMax{(2, 2), (2, 2), True, (0, 0)}.0, MakeVector.0)
Inputs types: [TensorType(float32, 4D), TensorType(int64, vector)]
Inputs shapes: [(100, 16, 8, 8), (4,)]
Inputs strides: [(4096, 256, 32, 4), (8,)]
Inputs values: ['not shown', array([100,  16,   4,   4], dtype=int64)]

Backtrace when the node is created:
  File "modules/layers/ConvolutionalPoolLayer.py", line 28, in propagate
    self.inputs = T.reshape(inputs, images_shape, ndim=4)

HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [6]:
net.compute_error_rate(cifar_test_stream)

0.2936